Like part 1, I think that the description of the task is rather more fiddly than the task itself.

We'll want the `knot_hash` function again:

In [1]:
from itertools import cycle, islice
from functools import reduce

In [2]:
def hash_once(listIn_ls, pos_i, length_i, skipSize_i=0):
    '''
    Apply the hash function once to an input list.
    
    Return a triple of the new list, new position, and new skip size.
    '''
    # Start by taking a cycle of the input list, and another
    # of indices we can use to update the list later
    listCycle_cc=cycle(listIn_ls)
    indexCycle_cc=cycle(range(len(listIn_ls)))
    
    # It'll also be useful to return a copy of the input list:
    listOut_ls=listIn_ls.copy()
    
    # Next, take a slice of the circular input list
    revSublist_ls=list(islice(listCycle_cc, pos_i, pos_i+length_i))
    # And reverse it:
    revSublist_ls.reverse()
    
    # And now replace the elements in listOut_ls with the
    # members of revSublist_ls. Use indexCycle_cc to get
    # them in the right place:
    
    for (i, j) in enumerate(islice(indexCycle_cc, pos_i, pos_i+length_i)):
        listOut_ls[j]=revSublist_ls[i]
        
    # The list has been updated; now need to return the appropriate
    # values, of the new list, the new position, and the new skip size
    
    newPos_i=(pos_i+length_i+skipSize_i)%len(listIn_ls)
    return (listOut_ls, newPos_i, skipSize_i+1)

In [3]:
def knot_hash(inputList_ls, lengths_ls):
    listOut_ls=inputList_ls.copy()
    pos_i=0
    skip_i=0
    
    for length_i in lengths_ls:
        (listOut_ls, pos_i, skip_i)=hash_once(listOut_ls, pos_i, length_i, skip_i)
    return listOut_ls

This time, the input list is the ascii values of the input string, plus the additional numbers 17, 31, 73, 47, 23. For the sparse hash, we want to run this 64 times. That's OK; python lets us multiply lists, so for a given input, we get:

In [4]:
# Start with one of the examples:

input_str='1,2,3'

sparseHash_ls=knot_hash(list(range(256)), 64*([ord(c) for c in input_str]+[17, 31, 73, 47, 23]))
sparseHash_ls[:10]

[5, 11, 95, 62, 101, 8, 254, 63, 51, 197]

Convert the sparse hash into blocks of 16:

In [5]:
[sparseHash_ls[16*x:16*(x+1)] for x in range(16)]

[[5, 11, 95, 62, 101, 8, 254, 63, 51, 197, 155, 31, 164, 190, 147, 6],
 [171, 98, 19, 20, 75, 159, 194, 207, 249, 91, 225, 78, 145, 230, 118, 224],
 [83, 87, 58, 244, 124, 241, 131, 201, 7, 114, 240, 204, 66, 47, 50, 252],
 [203, 213, 137, 127, 44, 165, 104, 234, 18, 25, 116, 173, 121, 209, 153, 138],
 [239, 13, 55, 157, 70, 189, 89, 196, 136, 243, 237, 210, 250, 246, 208, 59],
 [211, 215, 80, 40, 72, 135, 32, 54, 61, 21, 16, 238, 38, 77, 34, 184],
 [28, 160, 15, 181, 228, 103, 81, 229, 96, 37, 12, 217, 221, 36, 65, 235],
 [123, 109, 175, 187, 199, 198, 57, 113, 64, 0, 185, 1, 166, 152, 233, 253],
 [119, 67, 43, 143, 86, 191, 227, 149, 126, 105, 76, 146, 242, 120, 169, 156],
 [216, 162, 24, 219, 193, 129, 94, 48, 49, 10, 35, 82, 79, 174, 41, 22],
 [202, 108, 30, 115, 150, 154, 17, 88, 60, 26, 205, 148, 179, 128, 245, 45],
 [251, 142, 74, 102, 206, 182, 100, 3, 106, 56, 23, 99, 90, 222, 2, 172],
 [73, 158, 92, 161, 42, 93, 178, 141, 226, 200, 9, 33, 183, 180, 71, 132],
 [220,
  236,
  1

In [6]:
# For the dense hash, split the sparse hash into blocks of 16,
# and call XOR on the sets. Useful to use functools.reduce here

denseHash_ls=[reduce(lambda x, y:x^y, charList_ls)
              for charList_ls in [sparseHash_ls[16*x:16*(x+1)] for x in range(16)]]
denseHash_ls


[62, 251, 231, 138, 141, 130, 242, 153, 121, 3, 26, 74, 160, 177, 106, 157]

In [7]:
# Now convert to hex. Do a hacky conversion of 'x' to '0' to
# cover those cases where the hex form is only one character

hex_ls=[hex(x)[-2:].replace('x', '0') for x in denseHash_ls]
hex_ls

['3e',
 'fb',
 'e7',
 '8a',
 '8d',
 '82',
 'f2',
 '99',
 '79',
 '03',
 '1a',
 '4a',
 'a0',
 'b1',
 '6a',
 '9d']

and then concatenate them:

In [8]:
''.join(hex_ls)

'3efbe78a8d82f29979031a4aa0b16a9d'

That seems to accord with the example in the description. So let's create a single function to do the complete conversion:

In [9]:
def full_knot_hash(input_str):
    '''
    Do a full knot hash of the input string. Return the hashed version.
    '''
    
    sparseHash_ls=knot_hash(list(range(256)), 64*([ord(c) for c in input_str] + [17, 31, 73, 47, 23]))
    
    # For the dense hash, split the sparse hash into blocks of 16,
    # and call XOR on the sets. Useful to use functools.reduce here
    denseHash_ls=[reduce(lambda x, y:x^y, charList_ls)
              for charList_ls in [sparseHash_ls[16*x:16*(x+1)] for x in range(16)]]
    return ''.join([hex(x)[-2:].replace('x', '0') for x in denseHash_ls])
    

In [10]:
assert full_knot_hash('')=='a2582a3a0e66e6e86e3812dcb672a272'
assert full_knot_hash('AoC 2017')=='33efeb34ea91902bb2f59c9920caa6cd'
assert full_knot_hash('1,2,3')=='3efbe78a8d82f29979031a4aa0b16a9d'
assert full_knot_hash('1,2,4')=='63960835bcdc130f0b66d7ff4f6a5a8e'

Good! Now try with our input:

In [11]:
with open('data/day10.txt') as fIn:
    input_str=fIn.read().strip()

full_knot_hash(input_str)

'aff593797989d665349efe11bb4fd99b'